In [1]:
from typing import *

import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.python import keras

import os

from lib.dnn.utils import KerasTrainer, Optimizer
from lib.dnn.layers import Delta, MovingAverage


2021-11-04 16:47:55.935349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-04 16:47:55.935397: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
class Trial0Trainer(KerasTrainer):

	def __init__(self, seq_len, hidden_layers: List[int], *args, loss="binary_crossentropy", optimizer="adam", hidden_activation="relu", **kwargs):
		print(kwargs)
		super(Trial0Trainer, self).__init__(*args, **kwargs)
		self.__seq_len = seq_len
		self.__hidden_layers = hidden_layers
		self.__loss = loss
		self.__optimizer = optimizer
		self.__hidden_activation = hidden_activation

	def __get_currency_pairs(self, df: pd.DataFrame) -> List[Tuple[str, str]]:
		DELIMITER = "/"
		return [(pair.split(DELIMITER)[0], pair.split(DELIMITER)[1]) for pair in set(df["base_currency"] + DELIMITER + df["quote_currency"])]# TODO FIND A CLEAR WAY

	def __prepare_for_pair(self, sequence: np.ndarray, seq_len: int):
		data_len = sequence.shape[0] - seq_len
		X = np.zeros((data_len, seq_len))
		y = np.zeros((data_len,))
		for i in range(data_len):
			X[i] = sequence[i:i+seq_len]
			if sequence[i] > sequence[i-1]:
				y[i] = 1
			else:
				y[i] = 0
		return X, y
		
	def _prepare_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
		currency_pairs = self.__get_currency_pairs(data)
		
		X = []
		y = []
	
		for base_currency, quote_currency in currency_pairs:
			pair_sequence = data[data["base_currency"] == base_currency][data[data["base_currency"] == base_currency]["quote_currency"] == quote_currency]["c"].to_numpy() # TODO
			pair_X, pair_y = self.__prepare_for_pair(pair_sequence, self.__seq_len)
			X += list(pair_X)
			y += list(pair_y)
		
		
		return np.array(X), np.array(y)
			
	def _create_model(self) -> keras.Model:
		input_layer = keras.layers.Input(shape=self.__seq_len)
		
		output_layer = keras.layers.Dense(1, activation="softmax")
		
		if len(self.__hidden_layers) == 0:
			output_layer = output_layer(input_layer)
		else:
			hidden_layer = keras.layers.Dense(self.__hidden_layers[0], activation=self.__hidden_activation)(input_layer)
			for layer_size in self.__hidden_layers[1:]:
				hidden_layer = keras.layers.Dense(layer_size, activation=self.__hidden_activation)(hidden_layer)
			output_layer = output_layer(hidden_layer)
		
		model = keras.Model(inputs=input_layer, outputs=output_layer)
		
		return model
	
	def _compile_model(self, model):
		model.compile(optimizer=self.__optimizer, loss=self.__loss, metrics=["accuracy"])


In [4]:
class Trial0Optimizer(Optimizer):
	
	def __init__(self, max_power, max_depth, base=4, min_power=3):
		self.__max_power = max_power
		self.__max_depth = max_depth
		self.__base = base
		self.__min_power = min_power
		super(Trial0Optimizer, self).__init__()
	
	def __generate_possible_layers(self, layer_sizes, depth):
		if depth == 1:
			return [[size] for size in layer_sizes]
		
		layers = []
		returned_layers = self.__generate_possible_layers(layer_sizes, depth-1)
		layers += returned_layers
		for size in layer_sizes:
			for layer in returned_layers:
				layers += [[size] + layer]
		
		return layers
		
	def __generate_hidden_layers(self, base, min_power, max_power, max_depth) -> List[List[int]]:
		print("[+]Generating Hidden Layer Sizes...")
		
		layer_sizes = [base ** i for i in range(min_power, max_power)]
		
		return self.__generate_possible_layers(layer_sizes, max_depth)
		
	def _generate_param_values(self) -> Dict:
		return {
			"seq_len": [2**(i+2) for i in range(6)],
			"hidden_layers": self.__generate_hidden_layers(self.__base, self.__min_power, self.__max_power, self.__max_depth),
			"loss": ["binary_crossentropy"],
			"optimizer": ["adam", "sgd"],
			"hidden_activation": ["relu", "tanh"]
		}
		
	def _create_trainer(self, params) -> KerasTrainer:
		print("[+]Creating Trainer...")
		return Trial0Trainer(**params, export_path="temp/dummyModel.h5", data_path="Data/Minutes/AUD-CAD.csv", epochs=2)


In [5]:
MAX_POWER = 6
MIN_POWER = 3
BASE = 4


In [6]:
trainer = Trial0Trainer(10, [2, 3], data_path="Data/Minutes/AUD-CAD.csv", export_path="Model.h5", epochs=1)

In [8]:
optimizer = Trial0Optimizer(2, 1, 4, 1)


[+]Generating Hidden Layer Sizes...


In [9]:
optimizer.optimize()

[+]Starting Optimization...
[+]Optimizing Params: ['seq_len', 'hidden_layers', 'loss', 'optimizer', 'hidden_activation'] with Default Values: {}
[+]Optimizing Params: ['hidden_layers', 'loss', 'optimizer', 'hidden_activation'] with Default Values: {'seq_len': 4}
[+]Optimizing Params: ['loss', 'optimizer', 'hidden_activation'] with Default Values: {'seq_len': 4, 'hidden_layers': [4]}
[+]Optimizing Params: ['optimizer', 'hidden_activation'] with Default Values: {'seq_len': 4, 'hidden_layers': [4], 'loss': 'binary_crossentropy'}
[+]Optimizing Params: ['hidden_activation'] with Default Values: {'seq_len': 4, 'hidden_layers': [4], 'loss': 'binary_crossentropy', 'optimizer': 'adam'}
[+]Getting Loss for :{'seq_len': 4, 'hidden_layers': [4], 'loss': 'binary_crossentropy', 'optimizer': 'adam', 'hidden_activation': 'relu'}
[+]Creating Trainer...
{'export_path': 'temp/dummyModel.h5', 'data_path': 'Data/Minutes/AUD-CAD.csv', 'epochs': 2}
[+]Starting Training...
[+]Loading Data: Data/Minutes/AUD-CA

[+]Splitting Data...
[+]Model Summary
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2


    1/16150 [..............................] - ETA: 2:00:57 - loss: 0.5977 - accuracy: 0.2812

   32/16150 [..............................] - ETA: 26s - loss: 0.7519 - accuracy: 0.4688    

   66/16150 [..............................] - ETA: 24s - loss: 0.7447 - accuracy: 0.4754

...

6853/6922 [============================>.] - ETA: 0s - loss: 0.6920 - accuracy: 0.4744

6889/6922 [============================>.] - ETA: 0s - loss: 0.6921 - accuracy: 0.4745

6922/6922 [==============================] - 9s 1ms/step - loss: 0.6921 - accuracy: 0.4745


[+]Saving Model to temp/dummyModel.h5


TypeError: 'float' object is not subscriptable